In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss, accuracy_score
import optuna
import lightgbm as lgb
import sklearn
import numpy as np

c:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
# read the data from /input/train.csv
data = pd.read_csv("train.csv")

In [6]:
X = data.drop(['Class', 'id'], axis=1)
y = data['Class']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=42)

In [142]:
def objective(trial): 
    params = {
    'task': 'train', 
    'boosting_type':'goss',
    'objective':'binary',
    'metric':'binary_logloss', 
    'verbose':-10000000,
    'seed':42,
    'max_bin':32,
    'n_estimators':128,
    # 'n_estimators':trial.suggest_int("n_estimators", 800, 1200),
    'learning_rate':0.1,
    'feature_fraction':1.0,
    'bagging_fraction':1.0,
   # 'bagging_freq':trial.suggest_categorical("bagging_freq", [1]),
    'max_depth':trial.suggest_int("max_depth", 3, 10), # decrease this in the next round
    'num_leaves':trial.suggest_int("num_leaves",8, 24),
    'min_data_in_leaf':32,
    'min_gain_to_split':3.0,
}
    
    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=100)
    y_val_pred = model.predict_proba(X_val)
    logloss = log_loss(y_val, y_val_pred)
    return logloss

In [143]:
study = optuna.create_study(direction='minimize');
study.optimize(objective, n_trials=20);
optuna.visualization.plot_optimization_history(study)


[I 2023-03-12 21:21:09,653] A new study created in memory with name: no-name-08f19a9e-8e9d-4e90-bb8f-292afcebaf9d
c:\ProgramData\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning:

'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.



[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] min_data_in_leaf is set=32, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=32
[LightGBM] [Warning] min_gain_to_split is set=3.0, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=3.0
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[1]	valid_0's binary_logloss: 0.227215
[2]	valid_0's binary_logloss: 0.192475
[3]	valid_0's binary_logloss: 0.168232
[4]	valid_0's binary_logloss: 0.149424
[5]	valid_0's binary_logloss: 0.134054
[6]	valid_0's binary_logloss: 0.121352
[7]	valid_0's binary_logloss: 0.110643
[8]	valid_0's binary_logloss: 0.101457
[9]	valid_0's binary_logloss: 0.0935106
[10]	valid_0's binary_logloss: 0.0865868
[11]	valid_0's binary_logloss: 0.0805617
[12]	valid_0's binary_logloss: 0.0752919
[13]	valid_0's binary_logloss: 0.07062

[I 2023-03-12 21:21:10,052] Trial 0 finished with value: 0.033465811594983165 and parameters: {'max_depth': 7, 'num_leaves': 19}. Best is trial 0 with value: 0.033465811594983165.


[43]	valid_0's binary_logloss: 0.0344612
[44]	valid_0's binary_logloss: 0.0342999
[45]	valid_0's binary_logloss: 0.034186
[46]	valid_0's binary_logloss: 0.034099
[47]	valid_0's binary_logloss: 0.0340159
[48]	valid_0's binary_logloss: 0.033946
[49]	valid_0's binary_logloss: 0.0339009
[50]	valid_0's binary_logloss: 0.0338727
[51]	valid_0's binary_logloss: 0.0338247
[52]	valid_0's binary_logloss: 0.0337947
[53]	valid_0's binary_logloss: 0.0337793
[54]	valid_0's binary_logloss: 0.0337408
[55]	valid_0's binary_logloss: 0.0337102
[56]	valid_0's binary_logloss: 0.0336607
[57]	valid_0's binary_logloss: 0.0336193
[58]	valid_0's binary_logloss: 0.0335971
[59]	valid_0's binary_logloss: 0.0335606
[60]	valid_0's binary_logloss: 0.0335461
[61]	valid_0's binary_logloss: 0.0335407
[62]	valid_0's binary_logloss: 0.0335056
[63]	valid_0's binary_logloss: 0.033502
[64]	valid_0's binary_logloss: 0.0334894
[65]	valid_0's binary_logloss: 0.0334894
[66]	valid_0's binary_logloss: 0.0334894
[67]	valid_0's binar

c:\ProgramData\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning:

'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.



...

[84]	valid_0's binary_logloss: 0.0335077
[85]	valid_0's binary_logloss: 0.0335077
[86]	valid_0's binary_logloss: 0.0335077
[87]	valid_0's binary_logloss: 0.0335077
[88]	valid_0's binary_logloss: 0.033505
[89]	valid_0's binary_logloss: 0.033505
[90]	valid_0's binary_logloss: 0.033505
[91]	valid_0's binary_logloss: 0.033505
[92]	valid_0's binary_logloss: 0.033505
[93]	valid_0's binary_logloss: 0.033505
[94]	valid_0's binary_logloss: 0.033505
[95]	valid_0's binary_logloss: 0.033505
[96]	valid_0's binary_logloss: 0.0334989
[97]	valid_0's binary_logloss: 0.0334989
[98]	valid_0's binary_logloss: 0.0334977
[99]	valid_0's binary_logloss: 0.0334977
[100]	valid_0's binary_logloss: 0.0334977
[101]	valid_0's binary_logloss: 0.0334977
[102]	valid_0's binary_logloss: 0.0334977
[103]	valid_0's binary_logloss: 0.0334977
[104]	valid_0's binary_logloss: 0.0334977
[105]	valid_0's binary_logloss: 0.0334977
[106]	valid_0's binary_logloss: 0.0334977
[107]	valid_0's binary_logloss: 0.0334977
[108]	valid_0's 

c:\ProgramData\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning:

'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.



[1]	valid_0's binary_logloss: 0.227823
[2]	valid_0's binary_logloss: 0.193397
[3]	valid_0's binary_logloss: 0.16918
[4]	valid_0's binary_logloss: 0.150508
[5]	valid_0's binary_logloss: 0.135341
[6]	valid_0's binary_logloss: 0.122752
[7]	valid_0's binary_logloss: 0.112101
[8]	valid_0's binary_logloss: 0.10296
[9]	valid_0's binary_logloss: 0.0951164
[10]	valid_0's binary_logloss: 0.088281
[11]	valid_0's binary_logloss: 0.0822992
[12]	valid_0's binary_logloss: 0.0770222
[13]	valid_0's binary_logloss: 0.0724283
[14]	valid_0's binary_logloss: 0.0683336
[15]	valid_0's binary_logloss: 0.064758
[16]	valid_0's binary_logloss: 0.0616042
[17]	valid_0's binary_logloss: 0.0587684
[18]	valid_0's binary_logloss: 0.0562523
[19]	valid_0's binary_logloss: 0.0540376
[20]	valid_0's binary_logloss: 0.0520032
[21]	valid_0's binary_logloss: 0.0502031
[22]	valid_0's binary_logloss: 0.0485683
[23]	valid_0's binary_logloss: 0.0471352
[24]	valid_0's binary_logloss: 0.0458588
[25]	valid_0's binary_logloss: 0.0447

[I 2023-03-12 21:21:18,294] Trial 19 finished with value: 0.033393548506636686 and parameters: {'max_depth': 3, 'num_leaves': 12}. Best is trial 1 with value: 0.033310783087823544.


[103]	valid_0's binary_logloss: 0.0333935
[104]	valid_0's binary_logloss: 0.0333935
[105]	valid_0's binary_logloss: 0.0333935
[106]	valid_0's binary_logloss: 0.0333935
[107]	valid_0's binary_logloss: 0.0333935
[108]	valid_0's binary_logloss: 0.0333935
[109]	valid_0's binary_logloss: 0.0333935
[110]	valid_0's binary_logloss: 0.0333935
[111]	valid_0's binary_logloss: 0.0333935
[112]	valid_0's binary_logloss: 0.0333935
[113]	valid_0's binary_logloss: 0.0333935
[114]	valid_0's binary_logloss: 0.0333935
[115]	valid_0's binary_logloss: 0.0333935
[116]	valid_0's binary_logloss: 0.0333935
[117]	valid_0's binary_logloss: 0.0333935
[118]	valid_0's binary_logloss: 0.0333935
[119]	valid_0's binary_logloss: 0.0333935
[120]	valid_0's binary_logloss: 0.0333935
[121]	valid_0's binary_logloss: 0.0333935
[122]	valid_0's binary_logloss: 0.0333935
[123]	valid_0's binary_logloss: 0.0333935
[124]	valid_0's binary_logloss: 0.0333935
[125]	valid_0's binary_logloss: 0.0333935
[126]	valid_0's binary_logloss: 0.

In [144]:
study.best_trial.params

{'max_depth': 4, 'num_leaves': 18}

In [145]:
params = {
    'boosting_type':'goss',
    'objective':'binary',
    'metric':'binary_logloss', 
    'verbose':-10000000,
    'seed':42,
    'max_bin':32,
    'n_estimators':128,
    'learning_rate':0.1,
    'feature_fraction':1.0,
    'bagging_fraction':1.0,
    'min_data_in_leaf':80,
    'min_gain_to_split':3.0,
}

In [146]:
params.update(study.best_trial.params)

In [147]:
tuned_model = lgb.LGBMClassifier(**params)
tuned_model.fit(X_train, y_train, 
        eval_set=[(X_val, y_val)],
        eval_metric=['binary_logloss'],
        early_stopping_rounds=100) 

c:\ProgramData\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning:

'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.



[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] min_data_in_leaf is set=80, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=80
[LightGBM] [Warning] min_gain_to_split is set=3.0, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=3.0
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[1]	valid_0's binary_logloss: 0.227333
[2]	valid_0's binary_logloss: 0.192664
[3]	valid_0's binary_logloss: 0.168356
[4]	valid_0's binary_logloss: 0.149673
[5]	valid_0's binary_logloss: 0.134517
[6]	valid_0's binary_logloss: 0.121952
[7]	valid_0's binary_logloss: 0.111304
[8]	valid_0's binary_logloss: 0.102188
[9]	valid_0's binary_logloss: 0.0943018
[10]	valid_0's binary_logloss: 0.0874435
[11]	valid_0's binary_logloss: 0.0814099
[12]	valid_0's binary_logloss: 0.0761402
[13]	valid_0's binary_logloss: 0.07147

LGBMClassifier(bagging_fraction=1.0, boosting_type='goss', feature_fraction=1.0,
               max_bin=32, max_depth=4, metric='binary_logloss',
               min_data_in_leaf=80, min_gain_to_split=3.0, n_estimators=128,
               num_leaves=18, objective='binary', seed=42, verbose=-10000000)

In [148]:
preds = tuned_model.predict_proba(X_test)

In [149]:
testloss = log_loss(y_test, preds)
testloss

0.03567464847571605

In [150]:
train_preds = tuned_model.predict_proba(X_train)
trainloss = log_loss(y_train, train_preds)
trainloss

0.03338561169420938

In [151]:
# read the real test data from test.csv
test_data = pd.read_csv("test.csv")

In [62]:
X_testtest = test_data.drop(['id'], axis=1)
ID_test = test_data.id

In [63]:
preds = tuned_model.predict_proba(X_testtest)

In [64]:
preds

array([[9.99958735e-01, 4.12650535e-05],
       [9.98886839e-01, 1.11316119e-03],
       [9.99967033e-01, 3.29674258e-05],
       ...,
       [9.99862573e-01, 1.37426641e-04],
       [9.58702478e-01, 4.12975224e-02],
       [1.09207797e-02, 9.89079220e-01]])

In [65]:
preds=  preds[:,1]

In [66]:
preds

array([4.12650535e-05, 1.11316119e-03, 3.29674258e-05, ...,
       1.37426641e-04, 4.12975224e-02, 9.89079220e-01])

In [67]:
# Save test predictions to file
submission = pd.DataFrame({'id': ID_test,
                       'Class': preds})
submission.to_csv('submission_optuna_lgbm.csv', index=False)